# Modélisation du système bactérie/anticorps

Nadir Ait Kheddache, Idriss Alaoui Souliman, Fabien Benthami, Alma Draeger, Alexandre Janin

In [20]:
#Imports
import numpy as np
import ipywidgets as widgets
import IPython

In [21]:
#Variables globales
#probabilité d'avoir chaque type de cellule (0: vide, 1: bactérie, 2: globule blanc, 3: nutriments)
#la somme des probabilités doit être égale à 1
proba = [0.5, 0.1, 0.1, 0.3]

#probabilité qu'une cellule vide se transforme en nutriments
proba_apparition_nutriments = 0.1

#probabilité qu'un globule blanc dispairaisse à chaque tour
proba_apoptose = 0.01

#nombres de globules blancs qui apparaissent à chaque tour
nb_gb_apparition = 2

#nombres de GB nécessaires pour tuer une bactérie
nb_gb_necessaires = 2

#nombres de GB nécessaires pour qu'un GB apparaisse
nb_gb_reproduction = 4

#nombres de bactéries voisines à partir duquel la bactérie meurt 
nb_b_surpop = 5

#nombres de bactéries voisines à partir duquel le globule blanc meurt 
nb_b_necessaires = 5

#Symbole qui représente chaque valeur de cellule
cara = ['-', 'B', 'G', 'n']

In [22]:
#Paramètres
infiniCheckbox = widgets.Checkbox(value = False, description = "Infini", disabled = False)
display(infiniCheckbox)

Checkbox(value=False, description='Infini')

In [23]:
def matrice(l, h):
    """ int^2 -> int[][]
    largeur > 0, hauteur > 0
    Renvoie une matrice de 0 de l colonnes et h lignes"""
    
    return [[aleatoire() for x in range(l)] for y in range(h)]

In [24]:
def aleatoire():
    """float -> int
    Renvoie le type de cellule (0, 1, 2, 3) selon le nombre aléatoire tiré"""
    
    alea = np.random.rand()
    
    for i in range(len(proba)):
        p = proba[i]
        if alea <= p:
            return i
        else:
            alea -= p
            
    return 0

In [25]:
def printmatrice(M):
    """int[][] -> NoneType
    Affiche la matrice M"""
    for ligne in M:
        s = ""
        for cell in ligne:
            s += cara[cell] + ' '
        print(s + "\n")

In [26]:
def voisinage(M, x, y):
    """int[][] * int * int * bool-> int[]
    Renvoie la liste des voisins de la case (x, y).
    Si infini est False, on ne prend pas les voisins hors de la matrice"""
    v = [0, 0, 0, 0]
    
    ymax = len(M)
    xmax = len(M[0])
    
    infini = infiniCheckbox.value
    
    for i in [x - 1, x, x + 1]:
        if (0 <= i and i < xmax):
            for j in [y - 1, y, y + 1]:
                if (0 <= j and j < ymax):
                        if (i, j) != (x, y): #Si i, j n'est pas la case centrale
                            v[M[j][i]] += 1
    return v

In [27]:
def next_step_matrice(M):
    """int[][] -> int [][]
    Renvoie la matrice à l'étape suivante."""
    
    nv_M = list(M)
    
    for y in range(len(M)):
        for x in range(len(M[y])):
            nv_M[y][x] = next_step_cellule(M[y][x], voisinage(M, x, y))
            
    return nv_M

In [28]:
def next_step_cellule(c, voisins):
    """int * int[] -> int
    Renvoie l'état suivant d'une cellule c en fonction de son état actuel et de ses voisins."""
    
    vides = voisins[0]
    bacteries = voisins[1]
    globules = voisins[2]
    nutriments = voisins[3]
    
    
    if (c == 0): #La cellule est vide
        if (globules >= nb_gb_reproduction):
            return 2
        elif (np.random.rand() <= proba_apparition_nutriments):
            return 3
        else:
            return 0
        
    if (c == 1): #Bactérie
        if (globules >= nb_gb_necessaires):
            return 0
        elif (nutriments == 0):
            return 0
        elif(bacteries >= nb_b_surpop):
            return 0
        else:
            return 1
        
    if (c == 2): #Globule blanc
        if (bacteries >= nb_b_necessaires):
            return 0
        elif (np.random.rand() <= proba_apoptose):
            return 0
        else:
            return 2
        
    if (c == 3):
        if (bacteries > 0):
            return 1
        else:
            return 3

In [33]:
mat = matrice(16,8)

def next_step(b):
    global mat
    mat = next_step_matrice(mat)
    printmatrice(mat)

In [34]:
button = widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)

button.on_click(next_step)

display(button)

Button(description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

- B - B B G B B B G - n - - - - 

B G - B B B B B - B - G - G - G 

- - B - G n B - B - B - - - n n 

B B B - - - - - - B - n - - - - 

B B B - B B B B B B - G - - - - 

B - B B - B B B - - - n - n - - 

G B - - B - - G - n G G - - B B 

G - - B B B - G n - - G - B - - 

